In [1]:
import pulp
import turtle

# Fonction pour lire les données à partir d'un fichier texte
def read_data_from_txt(filename):
    with open(filename, "r") as f:
        lines = f.readlines()

    # Extraction des données du fichier texte
    N = int(lines[0].strip().split('=')[1].strip(';'))
    W = int(lines[2].strip().split('=')[1].strip(';'))
    type_tasks = list(map(int, lines[4].strip().split('=')[1].strip(';').strip()[1:-1].split(',')))
    A = list(map(int, lines[6].strip().split('=')[1].strip(';').strip()[1:-1].split(',')))
    e = list(map(float, lines[8].strip().split('=')[1].strip(';').strip()[1:-1].split(',')))
    lines[10].replace('],', ']§')
    P_str = lines[10].strip().split('=')[1].strip(';')[2:-1]  # Extract raw string
    P_str = P_str.replace('],', ']§')  # Replace ], with ]§
    P = [list(map(int, sublist.strip()[1:-1].split(',')))  # Convert to nested list of integers
         for sublist in P_str.split('§')]
    lines[12].replace('],', ']§')
    TT_str = lines[12].strip().split('=')[1].strip(';')[2:-1]  # Extract raw string
    TT_str = TT_str.replace('],', ']§')  # Replace ], with ]§
    TT = [list(map(int, sublist.strip()[1:-1].split(',')))  # Convert to nested list of integers
         for sublist in TT_str.split('§')]
    return N, W, type_tasks, A, e, P, TT

# Exemple d'utilisation
filename = input("Data file path : ")

N, W, type_tasks, A, e, P, TT = read_data_from_txt(filename)

# Initialisation du modèle d'optimisation
prob = pulp.LpProblem("Minimize_Makespan", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, w) for i in range(N) for w in range(W)), cat='Binary')
s = pulp.LpVariable.dicts("s", (i for i in range(N)), lowBound=0)
c = pulp.LpVariable.dicts("c", ((i, w) for i in range(N) for w in range(W)), lowBound=0)

# Fonction objectif
C_max = pulp.LpVariable("C_max", lowBound=0)
prob += C_max

# Définition de M (un grand nombre)
M = 10000

# Contraintes
for i in range(N):
    prob += pulp.lpSum(x[(i, w)] for w in range(W)) == 1  # Chaque tâche est assignée à un seul travailleur
    prob += s[i] >= A[i]  # Chaque tâche ne peut commencer avant son heure d'arrivée
    
    for w in range(W):
        prob += c[(i, w)] >= s[i] + (P[type_tasks[i]-1][w] / e[w]) * x[(i, w)] + TT[type_tasks[i]-1][w] * x[(i, w)]  # Calcul du temps de fin
        prob += C_max >= c[(i, w)]  # Mise à jour de C_max pour chaque temps de fin
        
        if i > 0:  # Contrainte de séquence pour les tâches sur le même travailleur
            for j in range(i):
                prob += s[i] >= c[(j, w)] - (1 - x[(i, w)]) * M

# Résolution du problème d'optimisation
prob.solve()

# Affichage des résultats de l'optimisation
print("Statut de l'optimisation:", pulp.LpStatus[prob.status])
print("Makespan minimum :", pulp.value(C_max))

# Création de l'animation turtle

# Fonction pour dessiner une tâche et la donner à un travailleur
def give_task(task_num, worker_num, start_time, end_time):
    turtle.penup()
    turtle.goto(20 + task_num * 150, 200 - worker_num * 100)
    turtle.pendown()
    turtle.fillcolor("lightblue")
    turtle.begin_fill()
    for _ in range(2):
        turtle.forward(100)
        turtle.right(90)
        turtle.forward(40)
        turtle.right(90)
    turtle.end_fill()
    turtle.penup()
    turtle.goto(70 + task_num * 150, 220 - worker_num * 100)
    turtle.write(f"Task {task_num + 1}\n{start_time}-{end_time}", align="center", font=("Arial", 10, "normal"))

# Fonction pour dessiner un travailleur
def draw_worker(worker_num):
    turtle.penup()
    turtle.goto(-300, 200 - worker_num * 100)
    turtle.pendown()
    turtle.shape("square")  # Forme prédéfinie pour le travailleur
    turtle.color("green")
    turtle.stamp()
    turtle.penup()
    turtle.goto(-280, 200 - worker_num * 100)
    turtle.pendown()
    turtle.write(f"Worker {worker_num + 1}", font=("Arial", 10, "normal"))

# Fonction pour déplacer le robot
def move_robot():
    turtle.penup()
    turtle.goto(-300 + (W - 1) * 150 / 2, 50)  # Placer le robot au milieu des travailleurs
    turtle.pendown()
    turtle.shape("circle")  # Forme prédéfinie pour le robot
    turtle.color("blue")

# Titre de l'interface graphique
turtle.title("Optimal Task Allocation")

# Titre de la répartition des tâches
turtle.penup()
turtle.goto(-300, 250)
turtle.pendown()
turtle.write("Task Allocation", align="center", font=("Arial", 14, "bold"))

# Affichage des travailleurs
for w in range(W):
    draw_worker(w)

# Affichage des tâches assignées aux travailleurs
for i in range(N):
    assigned = False
    for w in range(W):
        if pulp.value(x[(i, w)]) == 1:
            give_task(i, w, pulp.value(s[i]), pulp.value(c[(i, w)]))
            assigned = True
    if not assigned:
        print(f"Tâche {i+1} n'a pas été assignée à un travailleur.")

# Déplacer le robot
move_robot()

# Affichage du Makespan minimum
turtle.penup()
turtle.goto(-300, -250)
turtle.pendown()
turtle.write("Makespan minimum: " + str(pulp.value(C_max)), font=("Arial", 12, "normal"))

turtle.done()


Data file path :  C:\Users\malek\Desktop\malek\ENIT\data\datasemsys.txt


Statut de l'optimisation: Optimal
Makespan minimum : 66.0
